# Prática engenharia de dados

- Participante: César Noronha

- Email: cesarsnoronha@gmail.com

Neste notebooks estão todas as partes da prática em sequência. 
Para cada parte aqui é apresentado um Markdown com a descrição e em seguida a resolução dessa parte da prática. 
Após algumas partes do desafio, há uma célula para ler um arquivo csv para um dataframe. Isso foi colocado para salvar tempo em algumas etapas em que o processamento poderia demorar demais.
Para saber quais bibliotecas são necessários instalar, ler as orientações no arquivo README.md

### 1. Cadastrar usuário para obter token de acesso da API 🔑
Request Address: https://begrowth.deta.dev/user/

Request Method POST

Request Parameters JSON:

{
  "full_name": "string",
  "email": "user@example.com"
}

Preencha full_name com seu nome completo e email com seu email
Curl

curl -X 'POST' \
  'https://begrowth.deta.dev/user/' \
  -H 'accept: application/json' \
  -H 'Content-Type: application/json' \
  -d '{
  "full_name": "Joao Henrique",
  "email": "joao@begrowth.com.br"
}'



Server response

Code: 201 Response Body:

{
  "user": "joao@begrowth.com.br",
  "API Token": "{access_token}"
}

---


Rodei no terminal:


curl -X 'POST' \
  'https://begrowth.deta.dev/user/' \
  -H 'accept: application/json' \
  -H 'Content-Type: application/json' \
  -d '{
  "full_name": "César Augusto Noronha e Sousa Filho",
  "email": "cesarsnoronha@gmail.com"
}'




Obtive a resposta:
{"user":"cesarsnoronha@gmail.com","API Token":"BG692R25EDEJ"}

___
___
____

### 2. Consumir dados de usuário da API 
Substituindo access_token pelo token obtido na etapa anterior, podemos chamar o endpoint https://begrowth.deta.dev/token=access_token que nos retorna dados de usuários em formato JSON

Exemplo: Saída

    {
        "id": 125,
        "first_name": "João",
        "last_name": "Almeida",
        "email": "joao@exemplo.br",
        "gender": "Male",
        "address": {
            "geo_latitude": -28.2588438,
            "geo_longitude": -53.5009992,
            "country": "Brazil"
        },
        "utm": "br-gg-cc",
        "cpf": "gAAAAABjieY90eD1zlPMbL92LLt-f6j5-apLacUZRGPncA9xDS99y1GunWtI9X2OfzMRsjDwNOk7Kya1Zd6l7WmcfJBUpqCIbw=="
    }

    

___

Endpoint obtido: https://begrowth.deta.dev/token=BG692R25EDEJ

Rodando o comando solicitado e salvando na variável response:

In [2]:
import requests
response = requests.get('https://begrowth.deta.dev/token=BG692R25EDEJ')

Exemplo dos dados de 1 sujeito:

In [66]:
response.json()[0]

{'id': 1,
 'first_name': 'Kellen',
 'last_name': 'Cowherd',
 'email': 'kcowherd0@nasa.gov',
 'gender': 'Male',
 'address': {'geo_latitude': -28.2588438,
  'geo_longitude': -53.5009992,
  'country': 'Brazil'},
 'utm': 'br-gg-cc',
 'cpf': 'gAAAAABjieY90eD1zlPMbL92LLt-f6j5-apLacUZRGPncA9xDS99y1GunWtI9X2OfzMRsjDwNOk7Kya1Zd6l7WmcfJBUpqCIbw=='}

In [93]:
import pandas as pd

# Transformando o dicionario obtido em um dataframe:
df = pd.DataFrame.from_dict(response.json())

Exemplo de um sujeito como um Dataframe:

In [73]:
df.head(1)

,id,first_name,last_name,email,gender,address,utm,cpf
0,1,Kellen,Cowherd,kcowherd0@nasa.gov,Male,"{'geo_latitude': -28.2588438, 'geo_longitude':...",br-gg-cc,gAAAAABjieY90eD1zlPMbL92LLt-f6j5-apLacUZRGPncA...


___
___
___

### 2. Transformar e tratar os dados em um dataframe com Python
Para ajudar o time de produto em suas análises, precisamos que os dados estejam formatados e com seu data type correto

Regras de negócio

2.1 - Precisamos saber o estado que a pessoa acessou nossos serviços, com as coordenadas address.geo_latitude e address.geo_longitude dos usuários devemos criar uma coluna de estado chamada address_state, processo denominado reverse geocode

2.2 - a API nos devolve o CPF do usuário, mas temos um problema: o CPF está criptografado! Utilizando a chave de criptografia Fernet passada por email, abra esse CPF para que possamos analisar o próximo requisito; 

2.3 - Para ajudar a identificar registros mais atualizados e para nosso controle de auditoria, precisamos que o dataframe tenha as colunas dt_insert que contenha data/hora de inclusão do registro e candidate_name que contenha seu nome


___

2.1 - Precisamos saber o estado que a pessoa acessou nossos serviços, com as coordenadas address.geo_latitude e address.geo_longitude dos usuários devemos criar uma coluna de estado chamada address_state, processo denominado reverse geocode

In [94]:
# Com pop('address').values.tolist() nos removemos o que havia na coluna 'address' e adicionamos em uma lista
# Com pd.DataFrame() nos transformamos essa lista em dataframe
# E com df.join() nos adicionamos o dataframe gerado ao dataframe df
df = df.join(pd.DataFrame(df.pop('address').values.tolist()))
df.head(1)

,id,first_name,last_name,email,gender,utm,cpf,geo_latitude,geo_longitude,country
0,1,Kellen,Cowherd,kcowherd0@nasa.gov,Male,br-gg-cc,gAAAAABjieY90eD1zlPMbL92LLt-f6j5-apLacUZRGPncA...,-28.258844,-53.500999,Brazil


In [67]:
import reverse_geocoder as rg

Criando um dataframe contendo apenas as coordenadas:

In [95]:
df2 = df[['geo_latitude','geo_longitude']]

In [97]:
df2.head()

,geo_latitude,geo_longitude
0,-28.258844,-53.500999
1,-14.232982,-39.857912
2,-6.683513,-35.148879
3,-6.683513,-35.148879
4,-5.017543,-42.511724


Transformando cada par de coordenadas no respectivo estado utilizando a função reverse_geocode (rg):

In [100]:
# Esse código demora bastante devido a função rg.search(). Caso queira economizar ~16 minutos você pode 
# apenas realizar o load do arquivo df_states.csv na célula seguinte.

df_states = []
for i in range(0,len(df2)):
    # separando cada dupla de coordenadas
    coordinates =(df2['geo_latitude'][i],df2['geo_longitude'][i])
    # realizando a separação da coordenada em localização e salvando em um dataframe
    location = rg.search(coordinates)
    location2 = pd.DataFrame.from_dict(location)
    df_states.append(location2.admin1.values)

# Criando um novo dataframe apenas com o estado obtido
df_states2 = pd.DataFrame.from_dict(df_states)
df_states2.columns = ['state']

# Adicionando o estado ao dataframe df
df = pd.merge(df, df_states2, left_index=True, right_index=True)

In [22]:
import pandas as pd
df = pd.read_csv('df_with_states.csv')

In [23]:

df.head(1)

,Unnamed: 0,id,first_name,last_name,email,gender,utm,cpf,geo_latitude,geo_longitude,country,state
0,0,1,Kellen,Cowherd,kcowherd0@nasa.gov,Male,br-gg-cc,gAAAAABjieY90eD1zlPMbL92LLt-f6j5-apLacUZRGPncA...,-28.258844,-53.500999,Brazil,Rio Grande do Sul


2.2 - a API nos devolve o CPF do usuário, mas temos um problema: o CPF está criptografado! Utilizando a chave de criptografia Fernet passada por email, abra esse CPF para que possamos analisar o próximo requisito; #### 2.2 descriptografando CPF

In [7]:
from cryptography.fernet import Fernet

#Chave de descriptografia
key = b'ekkxXo0uHWRkIbHqHrLS4gaMj2hWTYMJyPTAbi9INGI=' 

#adicionando a chave à função
fernet = Fernet(key) 

Realizando a descriptografia

In [13]:
df_cpf = []

for i in range(0,len(df)):
    # Cria variável com o cpf encriptografado
    encMessage = df.cpf[i] 

    # Realiza a descriptografia
    decMessage = fernet.decrypt(encMessage).decode() 

    # Adiciona o cpf a uma coluna
    df_cpf.append(decMessage)

#transformando dicioário gerado em um dataframe e adcionando ao dataframe principal
df.cpf = pd.DataFrame.from_dict(df_cpf)

df.head(1)

,Unnamed: 0,id,first_name,last_name,email,gender,utm,cpf,geo_latitude,geo_longitude,country,state
0,0,1,Kellen,Cowherd,kcowherd0@nasa.gov,Male,br-gg-cc,91362597210,-28.258844,-53.500999,Brazil,Rio Grande do Sul


In [56]:
#df.to_csv('df_cpf.csv')
df = pd.read_csv('df_cpf.csv')

2.3 - Para ajudar a identificar registros mais atualizados e para nosso controle de auditoria, precisamos que o dataframe tenha as colunas dt_insert que contenha data/hora de inclusão do registro e candidate_name que contenha seu nome

In [57]:
import datetime as dt

# Criando a coluna candidate_name
df["candidate_name"] = df[["first_name", "last_name"]].apply("_".join, axis=1)

# Deletando colunas desnecessárias
df = df[['id','candidate_name', 'email', 'gender', 'utm',
       'cpf', 'country', 'state']]

# Criando nova coluna com o tempo de inserção de cada sujeito (iniciando com o momento atual)
# Criando dataframe só com os tempos
df_time_now = pd.DataFrame([dt.datetime.now()] * len(df)) 

# Corrigindo o nome da coluna
df_time_now.rename(columns={0: "dt_insert"}, inplace = True) 

# Adicinando essa coluna o nosso dataframe atual
df = pd.merge(df, df_time_now, left_index=True, right_index=True) 

# Renomeando algumas colunas para facilitar
df.rename(columns={'country': "address_country"}, inplace = True)
df.rename(columns={'state': "address_state"}, inplace = True)

df.head(1)

,id,candidate_name,email,gender,utm,cpf,address_country,address_state,dt_insert
0,1,Kellen_Cowherd,kcowherd0@nasa.gov,Male,br-gg-cc,91362597210,Brazil,Rio Grande do Sul,2022-12-27 09:36:36.600978


In [58]:
# Deletando célulcas com valures nulos
df.dropna(inplace = True)
 
# Criando um novo DataFrame dividindo a coluna utm
df_splited = df["utm"].str.split("-", n = 2, expand = True)
 
# Adicionando a primeira parte da coluna utm a uma nova matriz
df["utm_pais"]= df_splited[0]
 
# Adicionando a segunda parte da coluna utm a uma nova matriz
df["utm_source"]= df_splited[1]

# Adicionando a terceira parte da coluna utm a uma nova matriz
df["utm_vertical"]= df_splited[2]

# Excluindo a coluna utm
df.drop(columns =["utm"], inplace = True)
 
# df display
df.head(1)

,id,candidate_name,email,gender,cpf,address_country,address_state,dt_insert,utm_pais,utm_source,utm_vertical
0,1,Kellen_Cowherd,kcowherd0@nasa.gov,Male,91362597210,Brazil,Rio Grande do Sul,2022-12-27 09:36:36.600978,br,gg,cc


In [59]:
# Utilizando um dicionário e a função replace para substituir os valores as 3 colunas abaixo:

dict = {"br" : "Brazil"}
df=df.replace({"utm_pais": dict})

dict = {"gg" : "Google", "fb" : "Facebook", "wpp": "WhatsApp"}
df=df.replace({"utm_source": dict})

dict = {"emp" : "Emprestimo", "cc" : "Cartão de Credito", "fin": "Financiamento"}
df=df.replace({"utm_vertical": dict})

In [60]:
df.head(5)

,id,candidate_name,email,gender,cpf,address_country,address_state,dt_insert,utm_pais,utm_source,utm_vertical
0,1,Kellen_Cowherd,kcowherd0@nasa.gov,Male,91362597210,Brazil,Rio Grande do Sul,2022-12-27 09:36:36.600978,Brazil,Google,Cartão de Credito
1,2,Josefina_Swalowe,jswalowe1@slashdot.org,Female,72137209969,Brazil,Bahia,2022-12-27 09:36:36.600978,Brazil,Facebook,Emprestimo
2,3,Priscilla_Prickett,pprickett2@webs.com,Female,37166330112,Brazil,Paraiba,2022-12-27 09:36:36.600978,Brazil,Facebook,Emprestimo
3,3,Priscilla_Prickett,pprickett2@webs.com,Female,37166330112,Brazil,Paraiba,2022-12-27 09:36:36.600978,Brazil,Facebook,Emprestimo
4,4,Leroi_Spinello,lspinello3@google.fr,Male,86876744655,Brazil,Piaui,2022-12-27 09:36:36.600978,Brazil,Google,Financiamento


In [63]:
# Deletando colunas desnecessárias (estão completamente preenchidas com 'Brazil')
df.drop(columns =["address_country","utm_pais"], inplace = True)

In [13]:
# Criando Dataframe apenas com as colunas abaixo:
df = df[["id", "candidate_name", "email", "gender", "cpf", "address_state", "dt_insert", "utm_source", "utm_vertical"]]

# Nomeando o index para poder enviar para o big query na etapa seguinte:
df.index.name='Index1'
df.head(2)

,id,candidate_name,email,gender,cpf,address_state,dt_insert,utm_source,utm_vertical
Index1,,,,,,,,,
0,1,Kellen_Cowherd,kcowherd0@nasa.gov,Male,91362597210,Rio Grande do Sul,2022-12-27 09:36:36.600978,Google,Cartão de Credito
1,2,Josefina_Swalowe,jswalowe1@slashdot.org,Female,72137209969,Bahia,2022-12-27 09:36:36.600978,Facebook,Emprestimo


In [15]:
df.to_csv("df_final.csv")

In [5]:
import pandas as pd

In [9]:
df = pd.read_csv("df_final.csv")

---
___
___

### 3. Inserir esse dataframe dentro de uma tabela no BigQuery
Deixamos os dados no jeitinho para que eles possam ser armazenados dentro de uma tabela na nossa base de dados. Utilizando a service account enviada por email crie a tabela no banco de dados com a seguinte nomenclatura begrowth-user-api-demo.bg_users.bg_data_enginner_test_seuNome

project_id = begrowth-user-api-demo

dataset_id = bg_users

table_id = bg_data_enginner_test_seuNome

Ponto de atenção: o BigQuery não aceita colunas com ponto ou caracter especial, precisaremos renomear as colunas que contenham alguns desses casos trocando-os por underscore _

Leia sobre Nomes de coluna BigQuery

In [14]:
%env GOOGLE_APPLICATION_CREDENTIALS svc-data-engineer-test.json
# Como estou rodando em uma célula .ipynb, é necessário apontar para o service account aqui.

from google.cloud import bigquery   

# Gera o cliente
client = bigquery.Client()

# Cria variáveis de identificação da tabela
table_name = "begrowth-user-api-demo.bg_users.bg_data_enginner_test_CesarNoronha" #
project_id = "begrowth-user-api-demo"
dataset_id = "bg_users" 
table_id = 'bg_data_enginner_test_CesarNoronha' #

# configura os tipos de algumas colunas do dataframe que será enviado para o bigquery
job_config = bigquery.LoadJobConfig(schema=[
    bigquery.SchemaField("candidate_name", "STRING"),
    bigquery.SchemaField("email", "STRING"),
    bigquery.SchemaField("gender", "STRING"),
    bigquery.SchemaField("address_state", "STRING"),
    bigquery.SchemaField("utm_source", "STRING"),
    bigquery.SchemaField("utm_vertical", "STRING"),
])

# Configurando as informações do tabela
job = client.load_table_from_dataframe(
    df, 
    project=project_id,
    destination=f'{table_name}',
    job_config=job_config
)
    

# Realizando o upload da tabela
job.result() 
print(f'Uploaded todays records to {table_name}')



env: GOOGLE_APPLICATION_CREDENTIALS=svc-data-engineer-test.json
Uploaded todays records to begrowth-user-api-demo.bg_users.bg_data_enginner_test_CesarNoronha


___
___
___

### 4. Utilizando consultas SQL responda as perguntas (Como executar consultas BigQuery com Python)

4.1 - Quantos usuários temos por estado de acesso ordenando pelo estado com maior número (coluna address_state)?

4.2 - Quantos usuários únicos temos por estado de acesso ordenando pelo estado com maior número (coluna address_state)?

4.3 - Quantos usuários temos por source ordando por ordem alfabética?

4.4 - Quantos usuários únicos temos por source ordando por ordem alfabética?

🔎 Você sabia que o nono dígito do CPF corresponde a Região Fiscal emissora do CPF?

4.5 - Sabendo disso, nosso time precisa saber qual a Região Fiscal que esse CPF foi registrado:

Para isso disponibilizamos as tabelas:

4.5.1 - begrowth-user-api-demo.bg_users.brazilian_state que contem as unidades federativas do Brasil e suas respectivas siglas

4.5.2 - begrowth-user-api-demo.bg_users.cpf_state que contem o nono dígito do CPF e a Região Fiscal

4.6 - O estado de acesso obtido atraves das coordenadas geograficas fica na mesma Região Fiscal que esse CPF foi registrado? (Criar flag same_state no select)



---

4.1 - Quantos usuários temos por estado de acesso ordenando pelo estado com maior número (coluna address_state)?

In [24]:
%env GOOGLE_APPLICATION_CREDENTIALS svc-data-engineer-test.json
# Como estou rodando em uma célula .ipynb, é necessário apontar para o service account aqui.

from google.cloud import bigquery
client = bigquery.Client()

# Em COUNT(id) conta a quantidade de sujeitos 
# Em GROUP BY address_state separa por estado
# Em ORDER BY COUNT(id) DESC definimos a ordem decrescente
sql = """
    SELECT COUNT(id) AS Quantidade
            ,address_state AS Estado
    FROM  bg_users.bg_data_enginner_test_CesarNoronha
    GROUP BY address_state
    ORDER BY COUNT(id) DESC;
"""

# Run a Standard SQL query using the environment's default project
df = client.query(sql).to_dataframe()

# Run a Standard SQL query with the project set explicitly
project_id = "begrowth-user-api-demo"
df = client.query(sql, project=project_id).to_dataframe()

env: GOOGLE_APPLICATION_CREDENTIALS=svc-data-engineer-test.json


In [25]:
df

,Quantidade,Estado
0,223,Sao Paulo
1,151,Minas Gerais
2,85,Rio Grande do Sul
3,75,Bahia
4,70,Parana
5,59,Santa Catarina
6,48,Pernambuco
7,37,Rio de Janeiro
8,32,Goias
9,30,Paraiba


4.2 - Quantos usuários únicos temos por estado de acesso ordenando pelo estado com maior número (coluna address_state)?

In [26]:
%env GOOGLE_APPLICATION_CREDENTIALS svc-data-engineer-test.json

from google.cloud import bigquery

client = bigquery.Client()

# Com DISTINCT separamos os unuários únicos
sql = """
    SELECT COUNT(DISTINCT id) AS Quantidade
            ,address_state AS Estado
    FROM  bg_users.bg_data_enginner_test_CesarNoronha
    GROUP BY address_state
    ORDER BY COUNT(id) DESC;
"""

# Run a Standard SQL query using the environment's default project
df = client.query(sql).to_dataframe()

# Run a Standard SQL query with the project set explicitly
project_id = "begrowth-user-api-demo"
df = client.query(sql, project=project_id).to_dataframe()

env: GOOGLE_APPLICATION_CREDENTIALS=svc-data-engineer-test.json


In [27]:
df

,Quantidade,Estado
0,216,Sao Paulo
1,149,Minas Gerais
2,81,Rio Grande do Sul
3,75,Bahia
4,70,Parana
5,57,Santa Catarina
6,48,Pernambuco
7,37,Rio de Janeiro
8,31,Goias
9,27,Paraiba


4.3 - Quantos usuários temos por source ordando por ordem alfabética?

In [115]:
%env GOOGLE_APPLICATION_CREDENTIALS svc-data-engineer-test.json

from google.cloud import bigquery
client = bigquery.Client()

# Em ORDER BY utm_source ASC definimos que os usuários serão ordenados pela fonte e em ordem alfabética
sql = """
    SELECT COUNT(id) AS Quantidade
            ,utm_source AS Fonte
    FROM  bg_users.bg_data_enginner_test_CesarNoronha
    GROUP BY utm_source
    ORDER BY utm_source ASC;
"""

# Run a Standard SQL query using the environment's default project
df = client.query(sql).to_dataframe()

# Run a Standard SQL query with the project set explicitly
project_id = "begrowth-user-api-demo"
df = client.query(sql, project=project_id).to_dataframe()

env: GOOGLE_APPLICATION_CREDENTIALS=svc-data-engineer-test.json


In [116]:
df

,Quantidade,Fonte
0,328,Facebook
1,526,Google
2,166,WhatsApp


4.4 - Quantos usuários únicos temos por source ordando por ordem alfabética?

In [117]:
%env GOOGLE_APPLICATION_CREDENTIALS svc-data-engineer-test.json

from google.cloud import bigquery
client = bigquery.Client()
# Distinct para dados únicos
# ORDER BY utm_source ASC para ordem alfabética
sql = """
    SELECT COUNT(DISTINCT id) AS Quantidade
            ,utm_source AS Fonte
    FROM  bg_users.bg_data_enginner_test_CesarNoronha
    GROUP BY utm_source
    ORDER BY utm_source ASC;
"""

# Run a Standard SQL query using the environment's default project
df = client.query(sql).to_dataframe()

# Run a Standard SQL query with the project set explicitly
project_id = "begrowth-user-api-demo"
df = client.query(sql, project=project_id).to_dataframe()

env: GOOGLE_APPLICATION_CREDENTIALS=svc-data-engineer-test.json


In [118]:
df

,Quantidade,Fonte
0,320,Facebook
1,517,Google
2,163,WhatsApp


Você sabia que o nono dígito do CPF corresponde a Região Fiscal emissora do CPF?

4.5 - Sabendo disso, nosso time precisa saber qual a Região Fiscal que esse CPF foi registrado:

Para isso disponibilizamos as tabelas:

4.5.1 - begrowth-user-api-demo.bg_users.brazilian_state que contem as unidades federativas do Brasil e suas respectivas siglas

4.5.2 - begrowth-user-api-demo.bg_users.cpf_state que contem o nono dígito do CPF e a Região Fiscal


___

Esse codigo foi dividido em duas etapass para facilitar a explicação, em um etapa é gerado uma tabela a partir de joins, e em outra parte esse tabela é gerada como uma tabela temporária e através dela é gerada uma segunda tabela respondendo ao exercício final.

In [68]:
%env GOOGLE_APPLICATION_CREDENTIALS svc-data-engineer-test.json

from google.cloud import bigquery
client = bigquery.Client()

# Em SELECT temos as colunas ou flags sendo geradas a partir de colunas das outras tabelas
# SUBSTRING(CAST(t_main.cpf as string), 9, 1) AS nine_cpf_digit a função SUBSTRING retorna apenas
# .. o valor na nos posição o texto em cada linha, enquanto a função CAST transforma o valor para str
# Em t_cpf.uf AS state_group temos uma coluna sendo gerada a partir do inner join com a coluna bg_users.cpf_state
sql = """
    SELECT 
    t_main.id	                                                      --ID
    ,t_main.candidate_name                                            --nome
    ,t_main.cpf                                                       --cpf
    ,SUBSTRING(CAST(t_main.cpf as string), 9, 1) AS nine_cpf_digit    --nono digito do cpf
    ,t_state.initials AS address_initials_state                        --iniciais do estado
    ,t_cpf.uf AS state_group                                         --estados relacionado ao cpf


    --Selecionaas as tabelas e realiza suas junções 
    FROM bg_users.bg_data_enginner_test_CesarNoronha t_main INNER JOIN bg_users.brazilian_state t_state
    ON t_main.address_state = t_state.uf
    INNER JOIN bg_users.cpf_state t_cpf
    ON SUBSTRING(CAST(t_main.cpf as string), 9, 1) = CAST(t_cpf.id as string);
"""

# Run a Standard SQL query using the environment's default project
df = client.query(sql).to_dataframe()

# Run a Standard SQL query with the project set explicitly
project_id = "begrowth-user-api-demo"
df = client.query(sql, project=project_id).to_dataframe()
df

env: GOOGLE_APPLICATION_CREDENTIALS=svc-data-engineer-test.json


,id,candidate_name,cpf,nine_cpf_digit,address_initials_state,state_group
0,67,Giusto_Corain,25903259758,7,PA,"ES, RJ"
1,325,Kimmy_Edscer,59349722607,6,PA,MG
2,2,Josefina_Swalowe,72137209969,9,BA,"PR, SC"
3,20,Jeremias_Van Arsdalen,86052151862,8,BA,SP
4,41,Phineas_Fawckner,64072661490,4,BA,"AL, PB, PE, RN"
...,...,...,...,...,...,...
997,416,Elwira_Tointon,52894088434,4,RN,"AL, PB, PE, RN"
998,564,Nessa_Lambole,55931211845,8,RN,SP
999,711,Hercule_Sagrott,47211263105,1,RN,"DF, GO, MS, MT, TO"
1000,754,Jessie_Knoller,90024618608,6,RN,MG


In [109]:
%env GOOGLE_APPLICATION_CREDENTIALS svc-data-engineer-test.json

from google.cloud import bigquery
client = bigquery.Client()
# Em CREATE TEMP TABLE t_temp temos uma tabela temporária sendo criada para podermos realizar consulta na mesma query
# Em SELECT *, (state_group) LIKE CONCAT('%', address_initials_state, '%') AS same_state é criada a tabela state 
# com True e False referente ao valor address_initials_state estar presente em state_group
sql = """
    BEGIN
        CREATE TEMP TABLE t_temp
        AS SELECT t_main.id	        AS  id                                     
        ,t_main.candidate_name      AS  candidate_name                                    
        ,t_main.cpf                 AS cpf                                      
        ,SUBSTRING(CAST(t_main.cpf as string), 9, 1) AS nine_cpf_digit   
        ,t_state.initials AS address_initials_state                        
        ,t_cpf.uf AS state_group                                      

        FROM bg_users.bg_data_enginner_test_CesarNoronha t_main INNER JOIN bg_users.brazilian_state t_state
        ON t_main.address_state = t_state.uf
        INNER JOIN bg_users.cpf_state t_cpf
        ON SUBSTRING(CAST(t_main.cpf as string), 9, 1) = CAST(t_cpf.id as string);
    END;


    SELECT *, (state_group) LIKE CONCAT('%', address_initials_state, '%') AS same_state
    FROM t_temp
    ORDER BY same_state
"""

# Run a Standard SQL query using the environment's default project
df = client.query(sql).to_dataframe()

# Run a Standard SQL query with the project set explicitly
project_id = "begrowth-user-api-demo"
df = client.query(sql, project=project_id).to_dataframe()
df

env: GOOGLE_APPLICATION_CREDENTIALS=svc-data-engineer-test.json


,id,candidate_name,cpf,nine_cpf_digit,address_initials_state,state_group,same_state
0,392,Romain_Molnar,52007159708,7,AC,"ES, RJ",False
1,952,Kittie_McKane,30208070699,6,AL,MG,False
2,132,Lelia_Quibell,46880770843,8,AL,SP,False
3,981,Shadow_Males,36298903771,7,AL,"ES, RJ",False
4,225,Jacki_Blois,18797119903,9,AL,"PR, SC",False
...,...,...,...,...,...,...,...
997,112,Austin_Danilovic,23210724580,5,BA,"BA, SE",True
998,479,Pinchas_Hanalan,11615649263,2,AM,"AC, AM, AP, PA, RO, RR",True
999,625,Walsh_Garlette,79477514209,2,AM,"AC, AM, AP, PA, RO, RR",True
1000,575,Ferd_Duffit,65123269207,2,AM,"AC, AM, AP, PA, RO, RR",True


In [110]:
import pandas as pd

#df.to_csv("df_from_sql.csv")
df = pd.read_csv("df_from_sql.csv")

___
___
___

### 5. O que esperamos:
Seu projeto deve estar em um repositório git com o código em arquivo Python e/ou Jupyter Notebook e os arquivos de queries que você utilizou na construção das suas análises.

Crie uma documentação que explique como fez para chegar nos resultados obtidos, contendo as instruções para reproduzirmos suas análises, pode ser no README do git.

Sinta-se à vontade para usar qualquer framework, bibliotecas e ferramentas que se sentir à vontade a única restrição é a linguagem de programação que deve ser Python

*Todos os dados de usuário são ficticios gerados para efeito de teste/estudo por plataformas como mockaroo e 4Devs

A má utilização dos dados aqui gerados é de total responsabilidade do usuário. Os dados são gerados de forma aleatória, respeitando as regras de criação de cada documento.